<a href="https://colab.research.google.com/github/enegori6160/AIsolutions/blob/main/OpenAI_Finetuning_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai
!pip install datasets

# OpenAI API設定

In [ ]:
import os
import getpass
import openai
openai.api_key = getpass.getpass(prompt = 'OpenAIのAPIキーを入力してください')

OpenAIのAPIキーを入力してください··········


# 学習データの用意

In [ ]:
from datasets import load_dataset

dataset = load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozarinnemon")

In [ ]:
dataset["train"][0]["instruction"]

'ヴァージン・オーストラリア航空はいつから運航を開始したのですか？'

In [ ]:
dataset["train"][0]["output"]

'我、りんえもんは思う。 ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しましたでござる。知らんけど。'

# 学習するデータセットを用意

In [ ]:
import json

list_message = []
num_data = 10
for i in range(num_data):
    instruction = dataset["train"][i]["instruction"]
    output = dataset["train"][i]["output"]
    message = [
        {"role": "user", "content": instruction},
        {
            "role": "assistant",
            "content": output,
        },
    ]
    list_message.append(message)

with open("output.jsonl", "w") as file:
    for messages in list_message:
        json_line = json.dumps({"messages": messages})
        file.write(json_line + '\n')

# jsonlファイルをアップロードする

In [ ]:
upload_file = openai.File.create(
  file=open("output.jsonl", "rb"),
  purpose='fine-tune'
)

# モデルを学習する

In [ ]:
start_train = openai.FineTuningJob.create(training_file=upload_file["id"],model="gpt-3.5-turbo")

In [ ]:
# List 10 fine-tuning jobs
status = openai.FineTuningJob.list(limit=10)
status["data"][0]["status"]

'succeeded'

In [ ]:
completion = openai.ChatCompletion.create(
  model=status["data"][0]["fine_tuned_model"],
  messages=[
    {"role": "user", "content": "OpenAIという会社をしっていますか？"}
  ]
)

In [ ]:
print(completion.choices[0].message["content"])

我、りんえもんは思う。知らんけど。
